# Synthetic Data Generation: SDV vs MOSTLY AI Comparison

## Framework Comparison
This notebook compares two synthetic data generation approaches on a large-scale dataset:

- **SDV (Synthetic Data Vault)** - Business Source License
- **MOSTLY AI SDK** - Apache 2.0 License - Open Source

## Dataset & Objective
We'll use the **US Census Income dataset (10M records)** to:
- Compare training performance and generation speed
- Evaluate synthetic data quality using comprehensive metrics
- Assess privacy preservation capabilities
- Provide practical guidance for framework selection

## Key Takeaways
- Performance benchmarks on large-scale data
- Quality comparison metrics
- Privacy assessment results

In [ ]:
%uv pip install -U sdv mostlyai-qa 'mostlyai[local]'

# 1. Data Preparation

## Loading the Dataset
We'll use the US Census Income dataset with 10M records containing demographic, employment, and financial information - ideal for testing synthetic data generation at scale.

In [ ]:
import pandas as pd

# Load the US Census dataset (10M records) from remote Parquet file
# Note: This is a large dataset - initial load may take a few minutes
data = pd.read_parquet('https://mostly-public-tutorials.s3.eu-central-1.amazonaws.com/datasets/census/census_10_mil.parquet')

# Display basic info about the dataset
print(f"Dataset shape: {data.shape}")
print(f"Memory usage: {data.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print("\nFirst 5 rows:")
print(data.head())

| Age | Workclass         | FNLWGT | Education  | Education Num | Marital Status      | Occupation         | Relationship   | Race  | Sex    | Capital Gain | Capital Loss | Hours/Week | Native Country  | Income |
|-----|-------------------|--------|------------|----------------|----------------------|---------------------|----------------|-------|--------|---------------|---------------|-------------|------------------|--------|
| 39  | State-gov         | 77516  | Bachelors  | 13             | Never-married        | Adm-clerical        | Not-in-family  | White | Male   | 2174          | 0             | 40          | United-States     | <=50K  |
| 50  | Self-emp-not-inc  | 83311  | Bachelors  | 13             | Married-civ-spouse   | Exec-managerial     | Husband        | White | Male   | 0             | 0             | 13          | United-States     | <=50K  |
| 38  | Private           | 215646 | HS-grad    | 9              | Divorced             | Handlers-cleaners   | Not-in-family  | White | Male   | 0             | 0             | 40          | United-States     | <=50K  |
| 53  | Private           | 234721 | 11th       | 7              | Married-civ-spouse   | Handlers-cleaners   | Husband        | Black | Male   | 0             | 0             | 40          | United-States     | <=50K  |
| 28  | Private           | 338409 | Bachelors  | 13             | Married-civ-spouse   | Prof-specialty      | Wife           | Black | Female | 0             | 0             | 40          | Cuba              | <=50K  |


## Dataset Overview

The dataset contains 15 columns with mixed data types:
- **Numerical**: age, fnlwgt, education_num, capital_gain, capital_loss, hours_per_week
- **Categorical**: workclass, education, marital_status, occupation, relationship, race, sex, native_country, income

This combination of numerical and categorical data makes it ideal for testing both frameworks' capabilities.

In [ ]:
# Display column names and basic data types
print("Column names:")
print(data.columns.tolist())
print(f"\nData types:")
print(data.dtypes)
print(f"\nMissing values per column:")
print(data.isnull().sum())

## Train/Holdout Split

We split the data into:
- **Training Set (80% - 8M records)**: For model training
- **Holdout Set (20% - 2M records)**: For quality evaluation

This split ensures we can properly assess synthetic data quality against unseen real data.

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training/holdout sets
# Using stratified split would be better for classification tasks, but not critical here
# random_state=1 ensures reproducible results
train, holdout = train_test_split(
    data, 
    test_size=0.2,      # 20% for holdout evaluation
    random_state=1,     # Fixed seed for reproducibility
    shuffle=True        # Ensure random sampling
)

print(f"Training set: {train.shape[0]:,} records ({train.shape[0]/len(data)*100:.1f}%)")
print(f"Holdout set:  {holdout.shape[0]:,} records ({holdout.shape[0]/len(data)*100:.1f}%)")

# Verify the split maintains similar distributions
print(f"\nIncome distribution in training set:")
print(train['income'].value_counts(normalize=True))
print(f"\nIncome distribution in holdout set:")
print(holdout['income'].value_counts(normalize=True))

# 2. SDV Metadata Configuration

## Metadata Setup
SDV requires metadata to understand your data structure. We'll use auto-detection to identify column types (numerical vs categorical), then validate the configuration.

## Auto-Detecting Metadata
SDV can automatically detect column types from the data. The auto-detection correctly identifies our numerical and categorical columns.

In [30]:
from sdv.metadata import Metadata

# Auto-detect metadata from the training data
# Note: We wrap the DataFrame in a dict with table name 'table' as required by SDV
# Using only training data to avoid data leakage
metadata = Metadata.detect_from_dataframes({'table': train})

In [ ]:
# Display the auto-detected metadata
print('Auto-detected metadata structure:\n')
print(metadata)

# Show a summary of detected column types
table_metadata = metadata.to_dict()['tables']['table']['columns']
numerical_cols = [col for col, info in table_metadata.items() if info['sdtype'] == 'numerical']
categorical_cols = [col for col, info in table_metadata.items() if info['sdtype'] == 'categorical']

print(f"\n📊 Metadata Summary:")
print(f"Numerical columns ({len(numerical_cols)}): {numerical_cols}")
print(f"Categorical columns ({len(categorical_cols)}): {categorical_cols}")

In [ ]:
# Validate the metadata structure
try:
    metadata.validate()
    print("✅ Metadata validation passed")
except Exception as e:
    print(f"❌ Metadata validation failed: {e}")
    # You would fix metadata issues here if any exist

In [ ]:
# Validate that the metadata matches the actual data structure
try:
    metadata.validate_data(data=({'table': train}))  # Use train data for consistency
    print("✅ Data validation against metadata passed")
except Exception as e:
    print(f"❌ Data validation failed: {e}")
    # This would indicate mismatches between metadata and actual data

# 3. SDV: Training and Generation

## Gaussian Copula Synthesizer
We'll use SDV's Gaussian Copula Synthesizer, which models the statistical relationships between variables and generates synthetic data that preserves these relationships.

In [ ]:
import time
from sdv.single_table import GaussianCopulaSynthesizer

# Initialize the synthesizer with our metadata
# GaussianCopula is good for mixed data types and preserving correlations
synthesizer = GaussianCopulaSynthesizer(metadata)

print("🚀 Starting SDV training...")
print(f"Training on {len(train):,} records with {len(train.columns)} features")

start_time = time.time()

# Train the synthesizer on our training data
# This learns the statistical relationships between variables
synthesizer.fit(train)

end_time = time.time()
elapsed_minutes = (end_time - start_time) / 60

print(f"✅ SDV training completed in {elapsed_minutes:.2f} minutes")


In [44]:
print("🎲 Starting SDV synthetic data generation...")

start_time = time.time()

# Generate synthetic data with the same number of rows as original dataset
# You can adjust num_rows based on your needs
target_rows = len(data)  # Generate same size as original
sdv_synthetic_data = synthesizer.sample(num_rows=target_rows)

end_time = time.time()
elapsed_minutes = (end_time - start_time) / 60

print(f"✅ SDV generation completed in {elapsed_minutes:.2f} minutes")
print(f"⏱️  Generation rate: {target_rows / (end_time - start_time):,.0f} records/second")
print(f"📊 Generated {len(sdv_synthetic_data):,} synthetic records")

# Quick preview of generated data
print("\nFirst 5 synthetic records:")
print(sdv_synthetic_data.head())

🎲 Starting SDV synthetic data generation...
✅ SDV generation completed in 1.56 minutes
⏱️  Generation rate: 106,792 records/second
📊 Generated 10,000,000 synthetic records

First 5 synthetic records:
   age workclass  fnlwgt     education  education_num      marital_status  \
0   29   Private  201054       HS-grad             11            Divorced   
1   24   Private  114657  Some-college             10       Never-married   
2   30         ?  106626    Assoc-acdm              9  Married-civ-spouse   
3   41         ?  134349  Some-college              8  Married-civ-spouse   
4   26   Private  197185     Assoc-voc             14       Never-married   

       occupation   relationship   race     sex  capital_gain  capital_loss  \
0  Prof-specialty        Husband  White    Male         85331             0   
1           Sales  Not-in-family  White  Female         42103          4356   
2    Adm-clerical        Husband  White    Male             0            29   
3           Sales    

In [ ]:
import os

# Save SDV synthetic data
output_file = './data/sdv_synthetic_data.parquet'
sdv_synthetic_data.to_parquet(output_file, index=False)

# Get file size in MB
file_size_mb = os.path.getsize(output_file) / 1024**2

print(f"💾 SDV synthetic data saved to: {output_file}")
print(f"📁 File size: {file_size_mb:.1f} MB")


# 4. Mostly AI: Training and Generation

## Deep Learning Approach
Mostly AI uses advanced deep learning models optimized for tabular data. The SDK provides local training capabilities with configurable parameters for training time and privacy settings.

In [ ]:
from mostlyai.sdk import MostlyAI

# Initialize Mostly AI SDK for local training
# local=True means we'll train models locally rather than using cloud API
print("🔧 Initializing Mostly AI SDK...")
mostly = MostlyAI(local=True)
print("✅ Mostly AI SDK initialized successfully")

In [ ]:
print("🚀 Starting Mostly AI training...")
print(f"Training on {len(train):,} records with {len(train.columns)} features")

start_time = time.time()

# Configure and start training
# Mostly AI automatically detects column types and optimizes model architecture
g = mostly.train(
    config={
        "name": "US Census Income 10 million",
        "tables": [
            {
                "name": "census",
                "data": train,
                "tabularModelConfiguration": {
                    "max_training_time": 100,  # Limit training time (minutes)
                    # Optional: Add differential privacy
                    # 'differential_privacy': {
                    #     'max_epsilon': 5.0,      # Privacy budget
                    #     'delta': 1e-5,           # Privacy parameter
                    # }
                    # Optional: Model tuning
                    # "max_epochs": 50,
                    # "batch_size": 1024,
                },
                # Optional: Column-specific configurations
                # "columns": {
                #     "income": {"encode": "target"},  # Mark as target variable
                # }
            }
        ],
    },
    start=True,  # Start training immediately
    wait=True,   # Wait for completion before proceeding
)

end_time = time.time()
elapsed_minutes = (end_time - start_time) / 60

print(f"✅ Mostly AI training completed in {elapsed_minutes:.2f} minutes")

In [ ]:
print("🎲 Starting Mostly AI synthetic data generation...")

start_time = time.time()

# Generate synthetic data using the trained generator
# size parameter controls how many records to generate
target_rows = len(data)
sd = mostly.generate(g, size=target_rows)
mostlyai_synthetic_data = sd.data()

end_time = time.time()
elapsed_minutes = (end_time - start_time) / 60

print(f"✅ Mostly AI generation completed in {elapsed_minutes:.2f} minutes")
print(f"⏱️  Generation rate: {target_rows / (end_time - start_time):,.0f} records/second")
print(f"📊 Generated {len(mostlyai_synthetic_data):,} synthetic records")

# Quick preview of generated data
print("\nFirst 5 synthetic records:")
print(mostlyai_synthetic_data.head())

# Basic quality check
print(f"\nMissing values in synthetic data: {mostlyai_synthetic_data.isnull().sum().sum()}")

In [ ]:
# Save Mostly AI synthetic data for comparison
output_file = './data/mostlyai_synthetic_data.parquet'
mostlyai_synthetic_data.to_parquet(output_file, index=False)
file_size_bytes = os.path.getsize(output_file)
print(f"💾 Mostly AI synthetic data saved to: {output_file}")
print(f"📁 File size: {file_size_bytes / 1024**2:.1f} MB")

# 5. Quality Assessment and Comparison

## Evaluation Framework
We'll use Mostly AI's comprehensive QA framework to evaluate both synthetic datasets. The assessment includes:

- **Accuracy Metrics**: How well synthetic data preserves statistical distributions (univariate, bivariate, trivariate)
- **Similarity Analysis**: Comparison between training, holdout, and synthetic data
- **DCR Privacy Metrics**: Distance to Closest Record analysis for privacy assessment
- **Overall Quality Score**: Combined metric for synthetic data fidelity

### Key Privacy Metrics:
- **DCR Share**: Proportion of synthetic records that are closer to holdout than training data (higher = better privacy)
- **DCR Training**: Average distance from synthetic to closest training record (higher = better privacy)
- **Optimal DCR Share**: ~0.5 indicates good balance between utility and privacy

Let's compare the results from both frameworks:

In [ ]:
# Import and initialize the quality assessment framework
from mostlyai import qa

# Initialize logging to see detailed evaluation progress
qa.init_logging()
print("🔍 Quality assessment framework initialized")

In [ ]:
print("📊 Evaluating SDV synthetic data quality...")

# Load the SDV synthetic dataset
sdv_synthetic_data = pd.read_parquet('./data/sdv_synthetic_data.parquet')

# Run comprehensive quality assessment
# This compares synthetic data against training and holdout sets
report_path, metrics = qa.report(
    syn_tgt_data=sdv_synthetic_data,    # SDV synthetic data
    trn_tgt_data=train,                 # Original training data
    hol_tgt_data=holdout,               # Holdout data for validation
    max_sample_size_embeddings=10_000,  # Limit sample size for efficiency
    report_path='sdv_qa_report.html'    # HTML report output
)

print(f"📋 SDV Quality Report saved to: {report_path}")
print("\n📈 SDV Quality Metrics:")
print(metrics.model_dump_json(indent=4))

# Extract key metrics for comparison
sdv_accuracy = metrics.accuracy.overall
sdv_dcr_share = metrics.distances.dcr_share
sdv_dcr_training = metrics.distances.dcr_training
print(f"\n🎯 SDV Summary:")
print(f"   Overall Accuracy: {sdv_accuracy:.3f}")
print(f"   DCR Share: {sdv_dcr_share:.3f} (higher is better for privacy)")
print(f"   DCR Training: {sdv_dcr_training:.3f} (higher is better for privacy)")

In [ ]:
print(train.describe())

In [ ]:
from sklearn.preprocessing import StandardScaler

# Select numeric columns
numeric_cols = train.select_dtypes(include='number').columns

# Fit scaler on training data
scaler = StandardScaler()
train_scaled = train.copy()
train_scaled[numeric_cols] = scaler.fit_transform(train[numeric_cols])

holdout_scaled = holdout.copy()
holdout_scaled[numeric_cols] = scaler.transform(holdout[numeric_cols])

mostlyai_scaled = mostlyai_synthetic_data.copy()
mostlyai_scaled[numeric_cols] = scaler.transform(mostlyai_synthetic_data[numeric_cols])


In [45]:
print("📊 Evaluating Mostly AI synthetic data quality...")

# Load the Mostly AI synthetic dataset
mostlyai_synthetic_data = pd.read_parquet('./data/mostlyai_synthetic_data.parquet')

# Run comprehensive quality assessment for Mostly AI
report_path, metrics = qa.report(
    syn_tgt_data=mostlyai_synthetic_data,  # Mostly AI synthetic data
    trn_tgt_data=train,                    # Original training data
    hol_tgt_data=holdout,                  # Holdout data for validation
    max_sample_size_embeddings=10_000,     # Limit sample size for efficiency
    report_path='mostlyai_qa_report.html'  # HTML report output
)

print(f"📋 Mostly AI Quality Report saved to: {report_path}")
print("\n📈 Mostly AI Quality Metrics:")
print(metrics.model_dump_json(indent=4))

# Extract key metrics for comparison
mai_accuracy = metrics.accuracy.overall
mai_dcr_share = metrics.distances.dcr_share
mai_dcr_training = metrics.distances.dcr_training
print(f"\n🎯 Mostly AI Summary:")
print(f"   Overall Accuracy: {mai_accuracy:.3f}")
print(f"   DCR Share: {mai_dcr_share:.3f} (higher is better for privacy)")
print(f"   DCR Training: {mai_dcr_training:.3f} (higher is better for privacy)")

📊 Evaluating Mostly AI synthetic data quality...
[2025-07-07 13:34:18,433] INFO   : prepared training data for accuracy: (8000000, 15)
[2025-07-07 13:34:22,417] INFO   : prepared holdout data for accuracy: (2000000, 15)
[2025-07-07 13:36:10,966] INFO   : prepared synthetic data for accuracy: (10000000, 15)
[2025-07-07 13:36:11,559] INFO   : encode datasets for embeddings
[2025-07-07 13:36:12,303] INFO   : calculated embeddings: syn=(10000, 25), trn=(10000, 25), hol=(10000, 25)
[2025-07-07 13:36:12,304] INFO   : report accuracy and correlations
[2025-07-07 13:36:12,304] INFO   : calculate original data bins
[2025-07-07 13:36:21,451] INFO   : store original data bins
[2025-07-07 13:36:21,467] INFO   : calculate synthetic data bins
[2025-07-07 13:36:33,379] INFO   : calculate correlations
[2025-07-07 13:36:39,297] INFO   : calculate correlations
[2025-07-07 13:36:44,477] INFO   : calculated univariates for 15 columns in 1.35 seconds
[2025-07-07 13:36:52,777] INFO   : calculated bivariate 

/Users/kennethhamilton/Desktop/sdv-mostly-experiment/venv/lib/python3.10/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning:

divide by zero encountered in matmul

/Users/kennethhamilton/Desktop/sdv-mostly-experiment/venv/lib/python3.10/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning:

overflow encountered in matmul

/Users/kennethhamilton/Desktop/sdv-mostly-experiment/venv/lib/python3.10/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning:

invalid value encountered in matmul

/Users/kennethhamilton/Desktop/sdv-mostly-experiment/venv/lib/python3.10/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning:

divide by zero encountered in matmul

/Users/kennethhamilton/Desktop/sdv-mostly-experiment/venv/lib/python3.10/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning:

overflow encountered in matmul

/Users/kennethhamilton/Desktop/sdv-mostly-experiment/venv/lib/python3.10/site-packages/sklearn/decomposition/_base.py:148: 

[2025-07-07 13:39:15,775] INFO   : calculate and plot distances
[2025-07-07 13:39:15,775] INFO   : calculate distances
[2025-07-07 13:39:15,969] INFO   : calculated DCRs for data.shape=(10000, 25) and query.shape=(10000, 25) in 0.18s
[2025-07-07 13:39:16,145] INFO   : calculated DCRs for data.shape=(10000, 25) and query.shape=(10000, 25) in 0.18s
[2025-07-07 13:39:16,329] INFO   : calculated DCRs for data.shape=(10000, 25) and query.shape=(10000, 25) in 0.18s
[2025-07-07 13:39:16,331] INFO   : DCR Share: 50.1%, NNDR Ratio: 0.379 - ALL columns
[2025-07-07 13:39:16,370] INFO   : calculated DCRs for data.shape=(10000, 11) and query.shape=(10000, 11) in 0.04s
[2025-07-07 13:39:16,414] INFO   : calculated DCRs for data.shape=(10000, 11) and query.shape=(10000, 11) in 0.04s
[2025-07-07 13:39:16,454] INFO   : calculated DCRs for data.shape=(10000, 11) and query.shape=(10000, 11) in 0.04s
[2025-07-07 13:39:16,455] INFO   : DCR Share: 50.3%, NNDR Ratio: 0.981 - 11 columns [[0, 3, 4, 5, 10, 11, 

In [46]:
# Add a final comparison section
print("\n" + "="*60)
print("🏆 FINAL COMPARISON")
print("="*60)
print(f"SDV      - Accuracy: {sdv_accuracy:.3f}, DCR Share: {sdv_dcr_share:.3f}, DCR Training: {sdv_dcr_training:.3f}")
print(f"MostlyAI - Accuracy: {mai_accuracy:.3f}, DCR Share: {mai_dcr_share:.3f}, DCR Training: {mai_dcr_training:.3f}")
print("\nInterpretation:")
print("• Higher accuracy = better statistical fidelity")
print("• Higher DCR Share = better privacy preservation (more diverse synthetic records)")
print("• Higher DCR Training = better privacy preservation (synthetic records farther from training data)")
print("• DCR Share ~0.5 indicates good balance between utility and privacy")
print("• Check HTML reports for detailed analysis")


🏆 FINAL COMPARISON
SDV      - Accuracy: 0.738, DCR Share: 0.538, DCR Training: 0.285
MostlyAI - Accuracy: 0.979, DCR Share: 0.507, DCR Training: 0.008

Interpretation:
• Higher accuracy = better statistical fidelity
• Higher DCR Share = better privacy preservation (more diverse synthetic records)
• Higher DCR Training = better privacy preservation (synthetic records farther from training data)
• DCR Share ~0.5 indicates good balance between utility and privacy
• Check HTML reports for detailed analysis
